# Biosemi Active II recorded file streaming

Resample and Save

In [ ]:
# import mne
# import numpy as np
# from mne.channels import make_standard_montage
# import matplotlib.pyplot as plt
# from mne.datasets import eegbci

# raw = mne.io.read_raw_bdf("datasets/group1_block1.bdf")

# eegbci.standardize(raw)  # set channel names
# montage = make_standard_montage("biosemi64")

# raw.set_montage(montage, on_missing='ignore')

# raw_eeg = raw.copy().resample(512)

# raw_eeg.save("datasets/biosemi_SSVEP.fif", overwrite=True)

Load data for stream

In [17]:
import mne
import numpy as np
from mne.channels import make_standard_montage
import matplotlib.pyplot as plt
from mne.datasets import eegbci


raw_eeg = mne.io.read_raw_fif("datasets/biosemi_SSVEP.fif")

Opening raw data file datasets/biosemi_SSVEP.fif...
    Range : 0 ... 188927 =      0.000 ...   368.998 secs
Ready.


C:\Users\pipo_\AppData\Local\Temp\ipykernel_12412\2241404531.py:8: RuntimeWarning: This filename (datasets/biosemi_SSVEP.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_eeg = mne.io.read_raw_fif("datasets/biosemi_SSVEP.fif")


In [18]:
from pylsl import StreamInfo, StreamOutlet
import time
import mne

events = mne.find_events(raw_eeg, shortest_event = 0, verbose = False) 

select_ch = ['O1','Oz','PO3','POz','Pz']
raw_eeg = raw_eeg.pick(select_ch)

eeg_info = StreamInfo(name='SomSom', type='EEG', channel_count=raw_eeg.info['nchan'],
                      nominal_srate=raw_eeg.info['sfreq'], channel_format='float32')
eeg_outlet = StreamOutlet(eeg_info)

event_dict =  {'12Hz': 8,
'24Hz': 4,
'6Hz': 10,
'30Hz': 2,
'trial_begin': 1,
'trial_end': 20,
'end_exper': 40
}

id_to_key = {v: k for k, v in event_dict.items()}

def send_eeg_samples(raw_eeg, chunk_size=15):
    """ Send raw EEG data in chunks through LSL. """
    sfreq = raw_eeg.info['sfreq']  # Sampling frequency
    n_samples = len(raw_eeg.times)  # Total number of samples in the raw data
    data, _ = raw_eeg[:, :]  # Get all the EEG data as an array
    index_timestamp = 0


    for idx in range(0, n_samples, chunk_size):
        if idx > events[index_timestamp,0]:
            if  events[index_timestamp,2] in id_to_key:
                print("event", id_to_key[events[index_timestamp,2]])
                index_timestamp += 1 


        # Extract a chunk of `chunk_size` samples
        chunk = data[:, idx:idx + chunk_size]

        # Transpose to match LSL format: list of samples (n_samples x n_channels)
        eeg_outlet.push_chunk(chunk.T.tolist())

        time.sleep(chunk_size / sfreq)  # Sleep to maintain timing


if __name__ == '__main__':
    while True:
        # Send raw EEG data in chunks
        send_eeg_samples(raw_eeg)


event trial_begin
event 24Hz
event trial_end
event trial_begin
event 12Hz
event trial_end
event trial_begin
event 6Hz
event trial_end
event trial_begin
event 24Hz
event trial_end
event trial_begin


KeyboardInterrupt: 